# Exercise 3: Reasoning, Memory, Tool calling examples

In [1]:
from dotenv import load_dotenv
_ = load_dotenv()

## Reasoning in LLMs

In [3]:
from langchain_openai import ChatOpenAI

# Reasoning-style model: ask it to think step by step
model = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)

question = """
I want a 3‑day trip to Miami in March.
Suggest a rough plan (day 1–3) and explain your reasoning step by step.
"""

response = model.invoke(question)
print(response.content)

Sure! Here’s a rough 3-day itinerary for a trip to Miami in March, taking into account the city's vibrant culture, beautiful beaches, and exciting nightlife. 

### Day 1: Arrival and Beach Day

**Morning: Arrival in Miami**
- Check into your hotel. Choose a location in South Beach for easy access to the beach and nightlife.
- Grab breakfast at a local café, such as **The Front Porch Café**, known for its delicious brunch options.

**Afternoon: Beach Time**
- Head to **South Beach**. Spend the afternoon relaxing on the sandy shores, swimming, or participating in water sports like paddleboarding or jet skiing.
- Enjoy lunch at a beachside restaurant, such as **Nikki Beach**, which offers a great atmosphere and beach views.

**Evening: Explore Ocean Drive**
- Stroll along **Ocean Drive** to take in the Art Deco architecture and vibrant atmosphere.
- Have dinner at **Mango’s Tropical Café**, where you can enjoy live music and a lively ambiance.
- If you're up for it, explore the nightlife 

## Memory to maintain context

#### Without explicitly adding memory

In [25]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

print("=== No memory ===")
print(llm.invoke("I love New York city. Remember it.").content)
print()
print(llm.invoke("Do I love New York city?").content)  # often wrong / generic

=== No memory ===
Got it! You love New York City. If you have any specific questions or topics about NYC that you'd like to discuss, feel free to share!

I can't know your personal feelings, but many people love New York City for its vibrant culture, diverse neighborhoods, and endless activities. If you enjoy the energy, food, art, and opportunities the city offers, it's likely you have a fondness for it! What do you like most about New York City?


#### With memory added

In [38]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

print("=== With simple manual memory ===")
context = "I love New York city. Remember that about me.\n"
print(llm.invoke(context + "Say hello to that city.").content)
print()

# Re‑use the same context when asking the next question
print(llm.invoke(context + "Which city do I love?").content)

=== With simple manual memory ===
Hello, New York City! It's a vibrant place filled with energy, culture, and endless possibilities. Your love for this iconic city is noted! Whether it's the bustling streets, the stunning skyline, or the diverse neighborhoods, there's always something special about NYC.

You love New York City!


### Tool use

In [43]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain.agents import create_agent

# 1) Define a simple tool
@tool
def get_mock_weather(city: str) -> str:
    """Return fake weather for demo purposes."""
    return f"The weather in {city} is usually warm and sunny this week."

# 2) Build an agent that can call that tool
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

system_prompt = """
You are a helpful travel assistant.
Use the get_mock_weather tool when the user asks about weather.
Explain briefly what you did.
"""

agent = create_agent(
    model=model,
    tools=[get_mock_weather],
    system_prompt=system_prompt,
)

# 3) Ask a question; agent decides if it should call the tool
result = agent.invoke(
    {
        "messages": [
            {"role": "user", "content": "I'm visiting Miami. How's the weather this week?"}
        ]
    }
)

print(result["messages"][-1].content)

I checked the weather for Miami, and it looks like it's going to be warm and sunny this week. Enjoy your trip!
